# Drwosy Driver Detection

In [ ]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import playsound
import imutils
import dlib
import cv2

In [ ]:
modelPath = "/Users/yousefgamaleldin/NU/3rd Year/2nd Semster/Image Processing/DIP Project/shape_predictor_68_face_landmarks.dat"
sound_path = "/Users/yousefgamaleldin/NU/3rd%20Year/2nd%20Semster/Image%20Processing/DIP%20Project/Alarm.wav"

In [ ]:
def eyeAr(eye):
    # compute the euclidean distances between the two sets of vertical eye landmarks
    V1= dist.euclidean(eye[1], eye[5])
    V2= dist.euclidean(eye[2], eye[4])
    # compute the euclidean distance between the horizontal eye landmark
    H= dist.euclidean(eye[0], eye[3])
    # return the eye aspect ratio
    EAR= ((V1+V2)/(2*H))
    return EAR

In [ ]:
def sound_alarm(path):
    # play an alarm sound
    playsound.playsound(path)

In [ ]:
# initialize dlib's face detector
# the facial landmark predictor
fdetector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(modelPath)

In [ ]:
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold for to set off the
# alarm
EAR_thresh = 0.25
Frames_Thresh = 20
# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off
FC = 0
Alarm_ON = False

# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [ ]:
# start the video stream thread
vs = VideoStream(src=0).start()


# loop over frames from the video stream
while True:
    # grab the thread of video streaming and read it in the frame
    frame = vs.read()
    
    # Resize the video frame
    #frame = imutils.resize(frame, width=800)
    
    #Convert it to greyscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect faces in the grayscale frame
    faces = fdetector(gray)

    
    # loop over the face detections
    for faces in faces:
        
        # determine the facial landmarks for the face region
        shape = predictor(gray, faces)
        
        # convert the facial landmark (x, y)-coordinates to a NumPy array
        shape = face_utils.shape_to_np(shape)
        
        # Extract the left and right eye indexes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        
        #Use the indexes to compute the eye aspect ratio for both eyes
        leftEAR = eyeAr(leftEye)
        rightEAR = eyeAr(rightEye)
        
        # get the average the eye aspect ratio together for both eyes
        EAR = (leftEAR + rightEAR) / 2.0
        
        # compute the convex hull for the left and right eye, then
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        
        # visualize each of the eyes
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        
        # check to see if the eye aspect ratio is below the pre-defined threshold
        if EAR < EAR_thresh:
            FC += 1
            
            # And if so, increment the blink frame counter
            if FC >= Frames_Thresh:
                
                # if the alarm is off, turn it on
                if not Alarm_ON:
                    Alarm_ON = True
                    
                    # check to see if an alarm file was supplied,
                    # and if so, start a thread to have the alarm
                    # sound played in the background
                    sound = Thread(target=sound_alarm(sound_path))
                    sound.start()
                # draw an alarm on the frame
                cv2.putText(frame, "DROWSINESS ALERT!!", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        # otherwise, the eye aspect ratio is not below the pre-defined threshold 
        #so reset the counter and alarm
        else:
            FC = 0
            Alarm_ON = False

        # draw the computed eye aspect ratio on the frame to help
        # with debugging and setting the correct eye aspect ratio
        # thresholds and frame counters
        cv2.putText(frame, "EAR: {:.2f}".format(EAR), (300, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
 
    # show the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)